In [1]:
import os
import sys
sys.path.append(r'E:\毕设\work\utils')
import numpy
import argparse
import sys
import soundfile
import numpy as np
import librosa
import h5py
import time
import pandas as pd
from scipy import signal
import matplotlib.pyplot as plt
import logging

import config
from utilities import calculate_scalar, create_folder, read_audio

In [2]:
dataset_dir = "E:\\毕设"
workspace = "E:\\毕设\\work"
data_type = "test"
mini_data = False

sample_rate = config.sample_rate
window_size = config.window_size
overlap = config.overlap
mel_bins = config.mel_bins

def calculate_logmel(audio_path, sample_rate, extractor, n, audio_cuts):

    (audio, _) = read_audio(audio_path, target_fs=sample_rate)

    [cut1,cut2] = audio_cuts[n].split(',')
    audio = audio[int(float(cut1)*sample_rate):int(float(cut2)*sample_rate)]

    audio = audio / np.max(np.abs(audio))

    feature = extractor.transform(audio)

    return feature

class LogMelExtractor():
    def __init__(self, sample_rate, window_size, overlap, mel_bins):
        
        self.window_size = window_size
        self.overlap = overlap
        self.ham_win = np.hamming(window_size)
        
        self.melW = librosa.filters.mel(sr=sample_rate, 
                                        n_fft=window_size, 
                                        n_mels=mel_bins, 
                                        fmin=50., 
                                        fmax=sample_rate // 2).T
    
    def transform(self, audio):
    
        ham_win = self.ham_win
        window_size = self.window_size
        overlap = self.overlap
    
        [f, t, x] = signal.spectral.spectrogram(
                        audio, 
                        window=ham_win,
                        nperseg=window_size, 
                        noverlap=overlap, 
                        detrend=False, 
                        return_onesided=True, 
                        mode='magnitude') 
        x = x.T
            
        x = np.dot(x, self.melW)
        x = np.log(x + 1e-8)
        x = x.astype(np.float32)
        
        return x
    
def ExtractMEL(data_type):
    
    if data_type == 'development':
        audio_dir = dataset_dir
        meta_csv = os.path.join(workspace, 'train.xlsx')
        
    elif data_type == 'test':
        audio_dir = dataset_dir
        meta_csv = os.path.join(workspace, 'test.xlsx')
        
    hdf5_path = os.path.join(workspace, 'features', 'logmel', 
                             '{}.h5'.format(data_type))
        
    create_folder(os.path.dirname(hdf5_path))
    df = pd.DataFrame(pd.read_excel(meta_csv))
    
    audio_names = []
    audio_cuts = []

    if data_type == 'development':
        manually_verifications = []
        target_labels = []

    for row in df.iterrows():

        audio_name = row[1]['name']
        audio_names.append(audio_name)
        audio_cut = row[1]['cut']
        audio_cuts.append(audio_cut)

        if data_type == 'development':

            target_label = row[1]['label']
            manually_verification = row[1]['manually_verification']

            target_labels.append(target_label)
            manually_verifications.append(manually_verification)
            
    # Feature extractor
    extractor = LogMelExtractor(sample_rate=sample_rate,
                                window_size=window_size,
                                overlap=overlap,
                                mel_bins=mel_bins)

    # Write out to h5 file
    hf = h5py.File(hdf5_path, 'w')

    hf.create_dataset(
        name='feature',
        shape=(0, mel_bins),
        maxshape=(None, mel_bins),
        dtype=np.float32)

    calculate_time = time.time()
    bgn_fin_indices = []
    
    # Extract feature for audios
    for (n, audio_name) in enumerate(audio_names):

        print(n, audio_name)

        audio_path = os.path.join(audio_dir, audio_name)
        feature = calculate_logmel(audio_path, sample_rate, extractor, n, audio_cuts)

        print(feature.shape)

        # Write feature to hdf5
        bgn_indice = hf['feature'].shape[0]
        fin_indice = bgn_indice + feature.shape[0]

        hf['feature'].resize((fin_indice, mel_bins))
        hf['feature'][bgn_indice: fin_indice] = feature

        bgn_fin_indices.append((bgn_indice, fin_indice))
        
    # Write infos to hdf5
    hf.create_dataset(name='filename', 
                      data=[s.encode() for s in audio_names], 
                      dtype='S32')

    hf.create_dataset(name='bgn_fin_indices',
                      data=bgn_fin_indices,
                      dtype=np.int32)

    if data_type == 'development':

        hf.create_dataset(name='label', 
                          data=[s.encode() for s in target_labels], 
                          dtype='S32')

        hf.create_dataset(name='manually_verification',
                          data=manually_verifications,
                          dtype=np.int32)

    hf.close()

    print("Write out hdf5 file to {}".format(hdf5_path))
    print("Time spent: {} s".format(time.time() - calculate_time))
    hf.close()

ExtractMEL("development")
ExtractMEL("test")

0 乐器音频/中音笙C0263/C0263A2-1c1-1.wav
(117, 64)
1 乐器音频/二胡L0266/L0266A2-1d22-2.wav
(40, 64)
2 乐器音频/南音三弦T0295/T0295A2-1e1.wav
(21, 64)
3 乐器音频/G调梆笛C0237/C0237A2-1c4-1.wav
(180, 64)
4 乐器音频/小闷笛C0303/C0303A2-1c1-1.wav
(123, 64)
5 乐器音频/葫芦琴L0314/L0314A2-1d9-1.wav
(30, 64)
6 乐器音频/六角高胡L0292/L0292A2-1c3-2.wav
(26, 64)
7 乐器音频/高音板胡L0239/L0239A2-1d9-2.wav
(61, 64)
8 乐器音频/中音笙C0263/C0263A2-1c2-2.wav
(68, 64)
9 乐器音频/编铓D0298/D0298A2-1c1-2.wav
(14, 64)
10 乐器音频/拉祜族葫芦笙C0306/C0306A2-1c3-2.wav
(261, 64)
11 乐器音频/侗笛C0316/C0316A2-1d2-2.wav
(38, 64)
12 乐器音频/G调梆笛C0237/C0237A2-1d5-2.wav
(10, 64)
13 乐器音频/琵琶T0262/T0262A2-1c5-1.wav
(20, 64)
14 乐器音频/澜沧小三弦T0300/T0300A2-1c1-1.wav
(48, 64)
15 乐器音频/G调梆笛C0237/C0237A2-1d4-2.wav
(24, 64)
16 乐器音频/独弦琴T0317/T0317A2-1e2.wav
(11, 64)
17 乐器音频/二弦L0285/L0285A2-1e1.wav
(53, 64)
18 乐器音频/中虎D0273/D0273A2-1e1.wav
(9, 64)
19 乐器音频/俄比C0311/C0311A2-1c2-2.wav
(64, 64)
20 乐器音频/中音板胡L0240/L0240A2-1e5.wav
(52, 64)
21 乐器音频/扬琴T0267/T0267A2-1c4-1.wav
(14, 64)
22 乐器音频/牛铃D0299/D0299A2-1e2.wav
(9, 64)
23 乐

(101, 64)
187 乐器音频/二胡L0266/L0266A2-1c3-1.wav
(44, 64)
188 乐器音频/澜沧小三弦T0300/T0300A2-1d4-1.wav
(48, 64)
189 乐器音频/高音键笙C0243/C0243A2-1c1-2.wav
(61, 64)
190 乐器音频/俄比C0311/C0311A2-1d2-1.wav
(230, 64)
191 乐器音频/琵琶T0262/T0262A2-1c4-2.wav
(68, 64)
192 乐器音频/管子C0265/C0265A2-1c3-2.wav
(42, 64)
193 乐器音频/傈僳族奇奔T0302/T0302A2-1d1-1.wav
(79, 64)
194 乐器音频/六角高胡L0292/L0292A2-1c4-1.wav
(36, 64)
195 乐器音频/二胡L0266/L0266A2-1d8-1.wav
(31, 64)
196 乐器音频/中胡L0297/L0297A2-1c2-1.wav
(23, 64)
197 乐器音频/扬琴T0267/T0267A2-1d2-1.wav
(200, 64)
198 乐器音频/中阮T0260/T0260A2-1c2-1.wav
(63, 64)
199 乐器音频/唢呐C0296/C0296A2-1c2-2.wav
(55, 64)
200 乐器音频/二弦L0285/L0285A2-1e4.wav
(56, 64)
201 乐器音频/中音板胡L0240/L0240A2-1e2.wav
(52, 64)
202 乐器音频/牛角琴L0312/L0312A2-1b3-1.wav
(148, 64)
203 乐器音频/吐良C0308/C0308A2-1d2-2.wav
(14, 64)
204 乐器音频/G调梆笛C0237/C0237A2-1c6-2.wav
(129, 64)
205 乐器音频/编钟D0241/D0241A2-1c4-1.wav
(54, 64)
206 乐器音频/曲锣D0286/D0286A2-1e1.wav
(9, 64)
207 乐器音频/牛角琴L0312/L0312A2-1d9-2.wav
(145, 64)
208 乐器音频/中音加键唢呐C0259/C0259A2-1d1-2.wav
(26, 64)
209 

(64, 64)
373 乐器音频/洞巴C0304/C0304A2-1e1.wav
(537, 64)
374 乐器音频/编钟D0241/D0241A2-1c4-1.wav
(54, 64)
375 乐器音频/G调新笛C0281/C0281A2-1d8-1.wav
(19, 64)
376 乐器音频/中音加键唢呐C0259/C0259A2-1b1-2.wav
(185, 64)
377 乐器音频/埙C0283/C0283A2-1c2-1.wav
(39, 64)
378 乐器音频/二弦L0285/L0285A2-1e1.wav
(53, 64)
379 乐器音频/管子C0265/C0265A2-1c4-1.wav
(16, 64)
380 乐器音频/巴乌C0310/C0310A2-1c2-2.wav
(200, 64)
381 乐器音频/葫芦琴L0314/L0314A2-1c1-2.wav
(141, 64)
382 乐器音频/二胡L0266/L0266A2-1c3-2.wav
(42, 64)
383 乐器音频/扬琴T0267/T0267A2-1c3-1.wav
(146, 64)
384 乐器音频/编铓D0298/D0298A2-1e1.wav
(65, 64)
385 乐器音频/扬琴T0267/T0267A2-1c1-2.wav
(129, 64)
386 乐器音频/中音笙C0263/C0263A2-1c1-2.wav
(64, 64)
387 乐器音频/芦笙C0307/C0307A2-1d4-2.wav
(72, 64)
388 乐器音频/唢呐C0296/C0296A2-1d4-1.wav
(34, 64)
389 乐器音频/古筝T0255/T0255A2-1c2-2.wav
(46, 64)
390 乐器音频/高音板胡L0239/L0239A2-1c1-1.wav
(51, 64)
391 乐器音频/竹排琴D0315/D0315A2-1c3-2.wav
(12, 64)
392 乐器音频/琵琶T0262/T0262A2-1c1-1.wav
(43, 64)
393 乐器音频/吐良C0308/C0308A2-1c5-2.wav
(64, 64)
394 乐器音频/中胡L0297/L0297A2-1b3-2.wav
(42, 64)
395 乐器音频/巴乌C0

(70, 64)
558 乐器音频/竹排琴D0315/D0315A2-1e1.wav
(42, 64)
559 乐器音频/二胡L0266/L0266A2-1d7-1.wav
(20, 64)
560 乐器音频/傈僳族奇奔T0302/T0302A2-1e1.wav
(17, 64)
561 乐器音频/牛角琴L0312/L0312A2-1d2-2.wav
(148, 64)
562 乐器音频/古筝T0255/T0255A2-1c4-2.wav
(54, 64)
563 乐器音频/低音加键唢呐C0257/C0257A2-1b1-1.wav
(10, 64)
564 乐器音频/巴乌C0310/C0310A2-1d4-2.wav
(17, 64)
565 乐器音频/A调曲笛C0280/C0280A2-1c1-1.wav
(23, 64)
566 乐器音频/琵琶T0262/T0262A2-1c2-1.wav
(36, 64)
567 乐器音频/编钟D0241/D0241A2-1c4-2.wav
(54, 64)
568 乐器音频/高音键笙C0243/C0243A2-1e3.wav
(51, 64)
569 乐器音频/二胡L0266/L0266A2-1c2-1.wav
(42, 64)
570 乐器音频/云锣D0279/D0279A2-1b1-2.wav
(56, 64)
571 乐器音频/牛角琴L0312/L0312A2-1c3-2.wav
(112, 64)
572 乐器音频/中阮T0260/T0260A2-1c3-1.wav
(57, 64)
573 乐器音频/南音三弦T0295/T0295A2-1e1.wav
(21, 64)
574 乐器音频/琵琶T0262/T0262A2-1c3-1.wav
(113, 64)
575 乐器音频/管子C0265/C0265A2-1e4.wav
(35, 64)
576 乐器音频/二胡L0266/L0266A2-1c1-2.wav
(55, 64)
577 乐器音频/低音笙C0264/C0264A2-1c3-2.wav
(77, 64)
578 乐器音频/扬琴T0267/T0267A2-1c1-2.wav
(113, 64)
579 乐器音频/竹排琴D0315/D0315A2-1e1.wav
(42, 64)
580 乐器音频/高音键笙

(126, 64)
745 乐器音频/俄比C0311/C0311A2-1d2-2.wav
(196, 64)
746 乐器音频/中音加键唢呐C0259/C0259A2-1c2-1.wav
(62, 64)
747 乐器音频/三弦T0289/T0289A2-1c4-2.wav
(30, 64)
748 乐器音频/古筝T0255/T0255A2-1d6-1.wav
(85, 64)
749 乐器音频/中音加键唢呐C0259/C0259A2-1e2.wav
(38, 64)
750 乐器音频/独弦琴T0317/T0317A2-1e1.wav
(9, 64)
751 乐器音频/葫芦琴L0314/L0314A2-1c3-1.wav
(128, 64)
752 乐器音频/中音笙C0263/C0263A2-1c3-1.wav
(64, 64)
753 乐器音频/唢呐C0296/C0296A2-1c5-2.wav
(20, 64)
754 乐器音频/古筝T0255/T0255A2-1c1-1.wav
(76, 64)
755 乐器音频/德C0305/C0305A2-1c1-1.wav
(162, 64)
756 乐器音频/椰胡L0288/L0288A2-1e1.wav
(58, 64)
757 乐器音频/巴乌C0310/C0310A2-1c1-2.wav
(166, 64)
758 乐器音频/扁八角高胡L0291/L0291A2-1c1-2.wav
(36, 64)
759 乐器音频/扬琴T0267/T0267A2-1d4-1.wav
(187, 64)
760 乐器音频/六角高胡L0292/L0292A2-1c1-2.wav
(37, 64)
761 乐器音频/编钟D0241/D0241A2-1c1-1.wav
(190, 64)
762 乐器音频/中音板胡L0240/L0240A2-1c1-2.wav
(54, 64)
763 乐器音频/傈僳族奇奔T0302/T0302A2-1c3-1.wav
(30, 64)
764 乐器音频/柳琴T0261/T0261A2-1c5-1.wav
(36, 64)
765 乐器音频/编铓D0298/D0298A2-1c4-1.wav
(84, 64)
766 乐器音频/南音三弦T0295/T0295A2-1d2-1.wav
(51, 64)
7

...

(227, 64)
8637 乐器音频/葫芦琴L0314/L0314A2-1d1-2.wav
(188, 64)
8638 乐器音频/葫芦琴L0314/L0314A2-1d5-2.wav
(107, 64)
8639 乐器音频/葫芦琴L0314/L0314A2-1c1-1.wav
(133, 64)
8640 乐器音频/葫芦琴L0314/L0314A2-1c5-1.wav
(77, 64)
8641 乐器音频/葫芦琴L0314/L0314A2-1c4-2.wav
(115, 64)
8642 乐器音频/葫芦琴L0314/L0314A2-1c3-1.wav
(99, 64)
8643 乐器音频/葫芦琴L0314/L0314A2-1c3-2.wav
(113, 64)
8644 乐器音频/葫芦琴L0314/L0314A2-1d6-1.wav
(21, 64)
8645 乐器音频/葫芦琴L0314/L0314A2-1c2-2.wav
(186, 64)
8646 乐器音频/葫芦琴L0314/L0314A2-1c2-2.wav
(225, 64)
8647 乐器音频/葫芦琴L0314/L0314A2-1e1.wav
(414, 64)
8648 乐器音频/葫芦琴L0314/L0314A2-1c4-2.wav
(113, 64)
8649 乐器音频/葫芦琴L0314/L0314A2-1c2-1.wav
(226, 64)
8650 乐器音频/葫芦琴L0314/L0314A2-1c1-2.wav
(141, 64)
8651 乐器音频/葫芦琴L0314/L0314A2-1d8-2.wav
(1117, 64)
8652 乐器音频/葫芦琴L0314/L0314A2-1d5-1.wav
(88, 64)
8653 乐器音频/葫芦琴L0314/L0314A2-1c5-2.wav
(76, 64)
8654 乐器音频/葫芦琴L0314/L0314A2-1c3-1.wav
(113, 64)
8655 乐器音频/葫芦琴L0314/L0314A2-1c2-1.wav
(235, 64)
8656 乐器音频/葫芦琴L0314/L0314A2-1d6-1.wav
(23, 64)
8657 乐器音频/葫芦琴L0314/L0314A2-1d2-1.wav
(18, 64)
8658 乐器音频/葫

(11, 64)
8820 乐器音频/铙钹D0270/D0270A2-1e1.wav
(20, 64)
8821 乐器音频/铙钹D0270/D0270A2-1d1-1.wav
(52, 64)
8822 乐器音频/铙钹D0270/D0270A2-1b2-2.wav
(162, 64)
8823 乐器音频/铙钹D0270/D0270A2-1e1.wav
(19, 64)
8824 乐器音频/铙钹D0270/D0270A2-1e1.wav
(19, 64)
8825 乐器音频/铙钹D0270/D0270A2-1e1.wav
(19, 64)
8826 乐器音频/铙钹D0270/D0270A2-1e1.wav
(19, 64)
8827 乐器音频/铙钹D0270/D0270A2-1e1.wav
(19, 64)
8828 乐器音频/铙钹D0270/D0270A2-1b2-1.wav
(137, 64)
8829 乐器音频/铙钹D0270/D0270A2-1d4-2.wav
(44, 64)
8830 乐器音频/铙钹D0270/D0270A2-1e1.wav
(19, 64)
8831 乐器音频/铙钹D0270/D0270A2-1b1-1.wav
(129, 64)
8832 乐器音频/铙钹D0270/D0270A2-1d5-2.wav
(37, 64)
8833 乐器音频/铙钹D0270/D0270A2-1d2-2.wav
(6, 64)
8834 乐器音频/雷琴L0256/L0256A2-1e1.wav
(79, 64)
8835 乐器音频/雷琴L0256/L0256A2-1c1-2.wav
(37, 64)
8836 乐器音频/雷琴L0256/L0256A2-1c5-2.wav
(36, 64)
8837 乐器音频/雷琴L0256/L0256A2-1c1-1.wav
(43, 64)
8838 乐器音频/雷琴L0256/L0256A2-1d5-2.wav
(149, 64)
8839 乐器音频/雷琴L0256/L0256A2-1e2.wav
(61, 64)
8840 乐器音频/雷琴L0256/L0256A2-1e2.wav
(33, 64)
8841 乐器音频/雷琴L0256/L0256A2-1d5-2.wav
(119, 64)
8842 乐器音频/雷琴L0256

(47, 64)
9009 乐器音频/雷琴L0256/L0256A2-1c5-1.wav
(36, 64)
9010 乐器音频/雷琴L0256/L0256A2-1e2.wav
(123, 64)
9011 乐器音频/雷琴L0256/L0256A2-1c1-2.wav
(36, 64)
9012 乐器音频/雷琴L0256/L0256A2-1c4-1.wav
(46, 64)
9013 乐器音频/雷琴L0256/L0256A2-1c4-2.wav
(59, 64)
9014 乐器音频/雷琴L0256/L0256A2-1c2-1.wav
(55, 64)
9015 乐器音频/雷琴L0256/L0256A2-1c3-1.wav
(45, 64)
9016 乐器音频/雷琴L0256/L0256A2-1c5-2.wav
(38, 64)
9017 乐器音频/雷琴L0256/L0256A2-1c2-1.wav
(64, 64)
9018 乐器音频/马锣D0276/D0276A2-1b1-1.wav
(123, 64)
9019 乐器音频/马锣D0276/D0276A2-1b2-1.wav
(166, 64)
9020 乐器音频/马锣D0276/D0276A2-1b2-2.wav
(161, 64)
9021 乐器音频/高音板胡L0239/L0239A2-1c1-1.wav
(57, 64)
9022 乐器音频/高音板胡L0239/L0239A2-1c2-1.wav
(53, 64)
9023 乐器音频/高音板胡L0239/L0239A2-1d8-2.wav
(93, 64)
9024 乐器音频/高音板胡L0239/L0239A2-1e1.wav
(123, 64)
9025 乐器音频/高音板胡L0239/L0239A2-1d11-2.wav
(67, 64)
9026 乐器音频/高音板胡L0239/L0239A2-1d4-1.wav
(61, 64)
9027 乐器音频/高音板胡L0239/L0239A2-1c2-2.wav
(51, 64)
9028 乐器音频/高音板胡L0239/L0239A2-1c3-1.wav
(59, 64)
9029 乐器音频/高音板胡L0239/L0239A2-1e4.wav
(12, 64)
9030 乐器音频/高音板胡L0239/L0239A2-

(102, 64)
9191 乐器音频/高音键笙C0243/C0243A2-1c5-1.wav
(72, 64)
9192 乐器音频/高音键笙C0243/C0243A2-1c1-1.wav
(61, 64)
9193 乐器音频/高音键笙C0243/C0243A2-1e2.wav
(48, 64)
9194 乐器音频/高音键笙C0243/C0243A2-1c1-2.wav
(63, 64)
9195 乐器音频/高音键笙C0243/C0243A2-1c3-2.wav
(101, 64)
9196 乐器音频/高音键笙C0243/C0243A2-1c1-1.wav
(88, 64)
9197 乐器音频/高音键笙C0243/C0243A2-1c3-2.wav
(111, 64)
9198 乐器音频/高音键笙C0243/C0243A2-1c1-1.wav
(67, 64)
9199 乐器音频/高音键笙C0243/C0243A2-1d8-1.wav
(1058, 64)
9200 乐器音频/高音键笙C0243/C0243A2-1c4-2.wav
(106, 64)
9201 乐器音频/高音键笙C0243/C0243A2-1e2.wav
(45, 64)
9202 乐器音频/高音键笙C0243/C0243A2-1c4-2.wav
(82, 64)
9203 乐器音频/高音键笙C0243/C0243A2-1c4-1.wav
(82, 64)
9204 乐器音频/高音键笙C0243/C0243A2-1c4-2.wav
(112, 64)
9205 乐器音频/高音键笙C0243/C0243A2-1c3-2.wav
(107, 64)
9206 乐器音频/高音键笙C0243/C0243A2-1c1-1.wav
(69, 64)
9207 乐器音频/高音键笙C0243/C0243A2-1c1-1.wav
(64, 64)
9208 乐器音频/高音键笙C0243/C0243A2-1c4-1.wav
(108, 64)
9209 乐器音频/高音键笙C0243/C0243A2-1c4-1.wav
(69, 64)
9210 乐器音频/高音键笙C0243/C0243A2-1c1-1.wav
(84, 64)
9211 乐器音频/高音键笙C0243/C0243A2-1c1-2.wav
(61, 64)

(71, 64)
9367 乐器音频/高音键笙C0243/C0243A2-1c1-1.wav
(67, 64)
9368 乐器音频/高音键笙C0243/C0243A2-1c2-1.wav
(57, 64)
9369 乐器音频/高音键笙C0243/C0243A2-1c1-1.wav
(70, 64)
9370 乐器音频/高音键笙C0243/C0243A2-1c4-1.wav
(104, 64)
9371 乐器音频/高音键笙C0243/C0243A2-1c4-2.wav
(104, 64)
9372 乐器音频/高音键笙C0243/C0243A2-1c1-1.wav
(85, 64)
9373 乐器音频/高音键笙C0243/C0243A2-1c1-1.wav
(72, 64)
9374 乐器音频/高音键笙C0243/C0243A2-1d1-2.wav
(11, 64)
9375 乐器音频/高音键笙C0243/C0243A2-1c3-2.wav
(89, 64)
9376 乐器音频/高音键笙C0243/C0243A2-1c5-1.wav
(73, 64)
9377 乐器音频/高音键笙C0243/C0243A2-1c3-1.wav
(97, 64)
9378 乐器音频/高音键笙C0243/C0243A2-1c4-2.wav
(107, 64)
9379 乐器音频/高音键笙C0243/C0243A2-1e2.wav
(46, 64)
9380 乐器音频/高音键笙C0243/C0243A2-1c1-1.wav
(60, 64)
9381 乐器音频/高音键笙C0243/C0243A2-1c4-2.wav
(104, 64)
9382 乐器音频/高音键笙C0243/C0243A2-1c3-1.wav
(95, 64)
9383 乐器音频/高音键笙C0243/C0243A2-1c3-2.wav
(123, 64)
9384 乐器音频/高音键笙C0243/C0243A2-1c1-2.wav
(73, 64)
9385 乐器音频/高音键笙C0243/C0243A2-1c4-2.wav
(110, 64)
9386 乐器音频/高音键笙C0243/C0243A2-1d5-2.wav
(113, 64)
9387 乐器音频/高音键笙C0243/C0243A2-1c4-2.wav
(99, 64)